In [122]:
import pandas as pd
normalised = pd.read_excel("normalised_BZK.xlsx")
# remove Unnamed columns
normalised = normalised.loc[:,~normalised.columns.str.startswith('Unnamed:')]

In [123]:
def convert_and_normalise_dates(df):
    # Identify columns ending with 'Date'
    date_columns = [col for col in df.columns if col.endswith('Date')]
    
    for col in date_columns:
        # Generate the Normalised column name
        normalised_col = col + 'Normalised'
        
        # Ensure the Normalised column exists in the DataFrame
        if normalised_col in df.columns:
            def convert_date(row):
                if pd.isna(row[normalised_col]) and pd.notna(row[col]):
                    date_value = pd.to_datetime(row[col], errors='coerce', dayfirst=True, infer_datetime_format=True)
                    if pd.notna(date_value):
                        return date_value.strftime('%Y-%m-%d')
                return row[normalised_col]
            
            # Convert the date format to yyyy-mm-dd and update the Normalised column if empty
            df[normalised_col] = df.apply(convert_date, axis=1)
    
    return df

normalised = convert_and_normalise_dates(normalised)

In [ ]:
# Display only columns that contain 'Date' in their names
date_columns = [col for col in df.columns if 'Date' in col]
normalised[date_columns].head(50)

In [125]:
#create MaritalStatus column based on ExtraRemarks1 and ExtraRemarks2

def create_marital_status(df, check_list):
    # Iterate over the rows to populate the 'Marital Status' column
    for index, row in df.iterrows():
        if str(row['ApplicantExtraRemarks1']).lower() in check_list:
            df.at[index, 'MaritalStatus'] = row['ApplicantExtraRemarks1']
        elif str(row['ApplicantExtraRemarks2']).lower() in check_list:
            df.at[index, 'MaritalStatus'] = row['ApplicantExtraRemarks2']
    
    return df

marital_list = ['led.','ledig','led','ledig.','verh','verh.','verheiratet','verw.','verw','Wwe.','Witwe','gesch.','verwitwet']
normalised = create_marital_status(normalised, marital_list)


In [ ]:
normalised['MaritalStatus'].tail(20)

In [127]:
#create VictimDeathStatus column based on VictimExtraRemarks1 and VictimExtraRemarks2 and VictimExtraRemarks3

def create_death_status(df, check_list):
    # Iterate over the rows to populate the 'Marital Status' column
    df['VictimDeathStatus'] = ''
    for index, row in df.iterrows():
        if str(row['VictimExtraRemarks1']).lower() in check_list:
            df.at[index, 'VictimDeathStatus'] = row['VictimExtraRemarks1']
        elif str(row['VictimExtraRemarks2']).lower() in check_list:
            df.at[index, 'VictimDeathStatus'] = row['VictimExtraRemarks2']
        elif str(row['VictimExtraRemarks3']).lower() in check_list:
            df.at[index, 'VictimDeathStatus'] = row['VictimExtraRemarks3']
        elif str(row['VictimDeathDate']).lower() in check_list:
            df.at[index, 'VictimDeathStatus'] = row['VictimDeathDate']
    
    return df

death_list = ['verst.','verst','verstorben','verschollen','umgekommen']
normalised = create_death_status(normalised, death_list)

In [ ]:
normalised['VictimDeathStatus'].head(50)

In [129]:
existing_file_path = 'C:/Users/mahsa/Documents/Wiedergutmachung/BZK/BZK-InformationExtraction/BZK-InformationExtraction/normalised_BZK.xlsx'

with pd.ExcelWriter(existing_file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    normalised.to_excel(writer, sheet_name='final', index=False)
